# **Efficient Yelp API Calls - Breakfast - Denver**
- Steven Phillips
- December 21, 2022

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
## Load saved json yelp-api access file back to python dictionary
with open('/Users/steve/Documents/GitHub/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# Create an instance with api key
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [4]:
# set API call parameters 
LOCATION = 'Denver, CO'
TERM = 'Breakfast'

In [5]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_CO_breakfast.json"
JSON_FILE

'Data/results_in_progress_CO_breakfast.json'

In [6]:
# function to check if file already exists

def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [7]:
## Create a new empty json file (delete the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_CO_breakfast.json already exists. Deleting previous file...
[i] Data/results_in_progress_CO_breakfast.json not found. Saving empty list to new file.
- 0 previous results found.


190

In [8]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/190 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


For loop stopped after maximum number of api calls reached.

## **Convert .json file to dataframe**

In [9]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,kRiwvntcXKDPdtpYKcGPhg,onefold-denver,Onefold,https://s3-media4.fl.yelpcdn.com/bphoto/s4ncc0...,False,https://www.yelp.com/biz/onefold-denver?adjust...,924,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 39.744667, 'longitude': -104.969325}","[pickup, delivery]",$$,"{'address1': '1420 E 18th Ave', 'address2': No...",+13039540877,(303) 954-0877,1019.671672
1,gagUrh3806qc5hZ14F0Odw,denver-biscuit-company-denver-20,Denver Biscuit Company,https://s3-media1.fl.yelpcdn.com/bphoto/bxPN9s...,False,https://www.yelp.com/biz/denver-biscuit-compan...,4460,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 39.740384, 'longitude': -104.949098}","[pickup, delivery]",$$,"{'address1': '3237 E Colfax Ave', 'address2': ...",+13033777900,(303) 377-7900,1887.248967
2,ZVRZablopgmiT7Z04DJAeA,cafe-miriam-denver,Cafe Miriam,https://s3-media4.fl.yelpcdn.com/bphoto/O-4JfX...,False,https://www.yelp.com/biz/cafe-miriam-denver?ad...,266,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",5.0,"{'latitude': 39.7483556470418, 'longitude': -1...",[delivery],$$,"{'address1': '2217 E 21st Ave', 'address2': ''...",+17203259027,(720) 325-9027,1658.629367
3,IvTB-XXwW9iww3wqKxCd1A,fox-run-cafe-denver-2,Fox Run Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/QIujhx...,False,https://www.yelp.com/biz/fox-run-cafe-denver-2...,127,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 39.739881, 'longitude': -104.945506}","[pickup, delivery]",$$,"{'address1': '3550 E Colfax', 'address2': None...",+13035374584,(303) 537-4584,2168.512811
4,lpiXtr3y25zPlnS6VFFa7Q,stowaway-kitchen-denver,Stowaway Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/5vlnGg...,False,https://www.yelp.com/biz/stowaway-kitchen-denv...,360,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 39.75862, 'longitude': -104.98661}","[pickup, delivery]",$$,"{'address1': '2528 Walnut St', 'address2': '',...",+17206362374,(720) 636-2374,2916.460857


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,mdhCibtrtoe6niEx33GpIg,snarfs-sandwiches-denver-3,Snarf's Sandwiches,https://s3-media2.fl.yelpcdn.com/bphoto/S8va55...,False,https://www.yelp.com/biz/snarfs-sandwiches-den...,89,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 39.75717, 'longitude': -104.98767}","[delivery, pickup]",$$,"{'address1': '2425 Broadway', 'address2': '', ...",+13035243466,(303) 524-3466,2831.399889
996,m1KGLAk86fQBustesOm-uQ,the-broken-tee-grill-englewood,The Broken Tee Grill,https://s3-media3.fl.yelpcdn.com/bphoto/GQoBsg...,False,https://www.yelp.com/biz/the-broken-tee-grill-...,6,"[{'alias': 'breakfast_brunch', 'title': 'Break...",3.5,"{'latitude': 39.643209, 'longitude': -105.012727}",[delivery],$$,"{'address1': '2101 W Oxford Ave', 'address2': ...",+13037810378,(303) 781-0378,10889.576378
997,2pgeyAji_C3LG-LzIQ2WoQ,einstein-bros-bagels-denver-2,Einstein Bros. Bagels,https://s3-media2.fl.yelpcdn.com/bphoto/RVAFaT...,False,https://www.yelp.com/biz/einstein-bros-bagels-...,61,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",2.5,"{'latitude': 39.6787374, 'longitude': -104.972...",[delivery],$,"{'address1': '1213 E Evans Ave', 'address2': '...",+13037229300,(303) 722-9300,6318.683256
998,HgRXwPw65V2LVJj4CpQ0dA,ajinoya-ramen-denver,Ajinoya Ramen,https://s3-media1.fl.yelpcdn.com/bphoto/SLt_os...,False,https://www.yelp.com/biz/ajinoya-ramen-denver?...,311,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.5,"{'latitude': 39.691411403275815, 'longitude': ...",[delivery],$$,"{'address1': '1390 S Colorado Blvd', 'address2...",+17206388678,(720) 638-8678,5530.625472
999,BFo6UFnAm7MV8qwjcTXM-A,la-loma-castle-rock-castle-rock,La Loma | Castle Rock,https://s3-media1.fl.yelpcdn.com/bphoto/SDCTr0...,False,https://www.yelp.com/biz/la-loma-castle-rock-c...,444,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 39.42205689025087, 'longitude': -...",[],$$,"{'address1': '6361 Promenade Pkwy', 'address2'...",+13036329900,(303) 632-9900,35700.795147


### **Check data for and remove duplicate results**

In [10]:
# this cell not run, results in error for column types 'list' 
# check for duplicate results
# final_df.duplicated().sum()

In [11]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [12]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [13]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_CO_breakfast.csv.gz', compression='gzip',index=False)

### Final DataFrame final_df

In [14]:
final_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,kRiwvntcXKDPdtpYKcGPhg,onefold-denver,Onefold,https://s3-media4.fl.yelpcdn.com/bphoto/s4ncc0...,False,https://www.yelp.com/biz/onefold-denver?adjust...,924,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 39.744667, 'longitude': -104.969325}","[pickup, delivery]",$$,"{'address1': '1420 E 18th Ave', 'address2': No...",+13039540877,(303) 954-0877,1019.671672
1,gagUrh3806qc5hZ14F0Odw,denver-biscuit-company-denver-20,Denver Biscuit Company,https://s3-media1.fl.yelpcdn.com/bphoto/bxPN9s...,False,https://www.yelp.com/biz/denver-biscuit-compan...,4460,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 39.740384, 'longitude': -104.949098}","[pickup, delivery]",$$,"{'address1': '3237 E Colfax Ave', 'address2': ...",+13033777900,(303) 377-7900,1887.248967
2,ZVRZablopgmiT7Z04DJAeA,cafe-miriam-denver,Cafe Miriam,https://s3-media4.fl.yelpcdn.com/bphoto/O-4JfX...,False,https://www.yelp.com/biz/cafe-miriam-denver?ad...,266,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",5.0,"{'latitude': 39.7483556470418, 'longitude': -1...",[delivery],$$,"{'address1': '2217 E 21st Ave', 'address2': ''...",+17203259027,(720) 325-9027,1658.629367
3,IvTB-XXwW9iww3wqKxCd1A,fox-run-cafe-denver-2,Fox Run Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/QIujhx...,False,https://www.yelp.com/biz/fox-run-cafe-denver-2...,127,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 39.739881, 'longitude': -104.945506}","[pickup, delivery]",$$,"{'address1': '3550 E Colfax', 'address2': None...",+13035374584,(303) 537-4584,2168.512811
4,lpiXtr3y25zPlnS6VFFa7Q,stowaway-kitchen-denver,Stowaway Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/5vlnGg...,False,https://www.yelp.com/biz/stowaway-kitchen-denv...,360,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 39.75862, 'longitude': -104.98661}","[pickup, delivery]",$$,"{'address1': '2528 Walnut St', 'address2': '',...",+17206362374,(720) 636-2374,2916.460857
